In [18]:
# -*- coding: utf-8 -*-
# @Time    : 2025/04/03 
import os
import re
import json
import glob
import pandas as pd
import numpy as np
import jieba 

In [ ]:
# Load the MeetFresh user dictionary
jieba.load_userdict("MF_dict.txt")

In [8]:
# Define the constants

CORE_KEYWORDS = {
    '鲜芋仙', 'Meet Fresh', 'MeetFresh', '台湾美食', '甜品', 
    '芋圆', 'taro', '仙草', 'grass jelly', '奶茶', 'milk tea',
    '豆花', 'tofu pudding', '奶刨冰', 'milked shaved ice',
    '红豆汤', 'purple rice soup', '紫米粥', 'red bean soup',
    '2001 Coit Rd', 'Park Pavillion Center', '(972) 596-6088'
}



In [3]:
cont = pd.read_json('..\..\Data\processed\contents_cooked.json')

In [4]:
cont.head(2)

,note_id,user_id,title,note_body,tag_list,image_count,content_type_video,hot_note,post_time,last_update_time,scraped_time,liked_count,collected_count,comment_count,share_count,interaction_count
0,67d0d605000000002903d86a,576d3bde82ec3952ff40c5e1,有没有和我一样【吃茶三千】一喝一个不吱声的,连地址都不想写了[笑哭R]\n虽然环境还不错，颜值也不错\n但我感觉自己完全拔草了\n尝了m...,"达拉斯网红奶茶,达拉斯奶茶,达拉斯,达拉斯生活,达拉斯美食,达拉斯周边,达拉斯探店",1,0,0,2025-03-12 00:32:05,2025-03-12 00:32:06,2025-03-12 00:55:08.175,5,2,10,3,17
1,67c8960f000000002503ff5f,576d3bde82ec3952ff40c5e1,【达拉斯·玩】超治愈系手工体验！MUMU Garden,"📍301 W Parker Rd #208[话题]#, Plano, TX 75023（快乐...","208,达拉斯生活,达拉斯探店,达拉斯,达拉斯吃喝玩乐,达拉斯手工,达拉斯周末,达拉斯周边,...",14,0,0,2025-03-05 18:21:03,2025-03-05 18:21:04,2025-03-12 00:55:08.248,47,23,7,37,77


In [16]:
# slice the 1st row of cont
cont.iloc[0]

note_id                                        67d0d605000000002903d86a
user_id                                        576d3bde82ec3952ff40c5e1
title                                             有没有和我一样【吃茶三千】一喝一个不吱声的
note_body             连地址都不想写了[笑哭R]\n虽然环境还不错，颜值也不错\n但我感觉自己完全拔草了\n尝了m...
tag_list                      达拉斯网红奶茶,达拉斯奶茶,达拉斯,达拉斯生活,达拉斯美食,达拉斯周边,达拉斯探店
image_count                                                           1
content_type_video                                                    0
hot_note                                                              0
post_time                                           2025-03-12 00:32:05
last_update_time                                    2025-03-12 00:32:06
scraped_time                                 2025-03-12 00:55:08.175000
liked_count                                                           5
collected_count                                                       2
comment_count                                                   

In [ ]:
# combine the title and note_body into a single string
def process_content(note):
    return note['title'] + ' ' + note['note_body']

#  showcase the first row of the dataframe
sample_note = cont.iloc[0]
combined_text = process_content(sample_note)

# clean the text by removing hashtags, mentions, URLs, new lines, tabs
def clean_text(text):
    text = re.sub(r'#\S+', '', text)  # remove topic hashtags
    text = re.sub(r'@\S+', '', text)  # remove user mentions
    text = re.sub(r'http\S+', '', text)  # remove URLs
    text = re.sub(r'www\S+', '', text)  # remove URLs
    # text = re.sub(r'\[.*?\]', '', text)  # remove emoji tags
    text = re.sub(r'\t', ' ', text)     # tab to space  
    text = re.sub(r'\n', ' ', text)     # new line to space
    text = re.sub(r'\s+', ' ', text)    # multiple spaces to single space
    return text.strip()


cleaned_text = clean_text(combined_text)

In [20]:
cleaned_text

'有没有和我一样【吃茶三千】一喝一个不吱声的 连地址都不想写了 虽然环境还不错，颜值也不错 但我感觉自己完全拔草了 尝了mango茶sample：仿佛化掉的芒果冰棍 冻顶乌龙：茶味白开水 又见柠檬挞：矮子里的将军，青柠汁绿茶+cream，虽然算清新派里能喝的，但依旧偏寡淡 松针厚奶：茶香精味重+依旧白开水 \t 是今天员工问题？还是达拉斯这家店问题？还是吃茶三千就这个风格？为什么值得排这么久的队呢…想不通啊想不通～'

In [ ]:
# enable jieba , and use the accurate mode 
jieba.enable_paddle()
jieba.set_dictionary('..\..\Data\processed\dict.txt.big')
jieba.initialize()